In [1]:
import pandas as pd

## Nine for news urls

In [6]:
nine_for_news_url_string = 'https://www.ninefornews.nl/page/13/; https://www.ninefornews.nl/europarlementarier-geeft-in-3-minuten-3-redenen-waarom-wereldwijd-pandemieverdrag-gevaarlijk-is/; https://www.ninefornews.nl/world-economic-forum-kondigt-zomer-davos-aan-en-hier-vindt-de-bijeenkomst-plaats/; https://www.ninefornews.nl/russische-soldaten-doen-opmerkelijke-uitspraken-over-oorlog-in-oekraine/; https://www.ninefornews.nl/harde-uithaal-naar-groene-gek-timmermans-hele-gevaarlijke-man/; https://www.ninefornews.nl/zo-hebben-ze-de-pandemie-nog-2-jaar-gerekt-dit-is-een-groot-schandaal/; https://www.ninefornews.nl/brisante-onthullingen-de-hufterigheid-van-rutte-cum-suis-kent-geen-grenzen/; https://www.ninefornews.nl/kaag-kiest-er-bewust-voor-om-de-nederlandse-bevolking-armer-te-maken/; https://www.ninefornews.nl/verslagenheid-na-aannemen-dictatoriale-pandemiewet-moeilijke-moeilijke-tijden-komen-eraan/; https://www.ninefornews.nl/instagram-influencer-verheerlijkt-euthanasie-heb-je-er-zin-in/; https://www.ninefornews.nl/timmermans-aangepakt-trek-die-verschrikkelijke-nederland-op-slot-wet-in/; https://www.ninefornews.nl/thierry-baudet-waarschuwt-ze-laten-nadrukkelijk-de-mogelijkheid-voor-klimaatlockdowns-open/; https://www.ninefornews.nl/eerste-kamerlid-christenunie-gaat-zonder-enig-benul-over-omstreden-pandemiewet-stemmen/; https://www.ninefornews.nl/fel-verzet-tegen-pandemiewet-geef-onze-vrijheid-niet-weg/; https://www.ninefornews.nl/schrijver-hier-heeft-het-duivelsgebroed-zich-in-nederland-genesteld/; https://www.ninefornews.nl/amerikaanse-acteur-doet-schokkend-boekje-open-over-luguber-ritueel-dat-hij-zag/; https://www.ninefornews.nl/fvd-stelt-kamervragen-over-bilderbergconferentie-waarom-krijgt-de-kamer-niets-te-horen/; https://www.ninefornews.nl/de-boeren-moeten-weg-want-rutte-wil-iets-heel-speciaals-samen-met-het-world-economic-forum/; https://www.ninefornews.nl/gevaccineerden-zullen-over-2-jaar-dood-zijn-zegt-deze-arts/; https://www.ninefornews.nl/europarlementarier-dit-is-waarom-de-who-bestempeld-moet-worden-als-terreurorganisatie/; https://www.ninefornews.nl/arts-over-nederlanders-die-heulen-met-perverse-terreur-van-von-der-leyen-en-schwab-pak-ze-aan/; https://www.ninefornews.nl/rusland-dreigt-onze-f-16s-uit-de-lucht-te-schieten-boven-oekraine/; https://www.ninefornews.nl/oekraine-doet-schokkende-bekentenis-die-is-genegeerd-door-westerse-media/; https://www.ninefornews.nl/nieuwe-pensioenwet-gekraakt-wanneer-zoiets-in-een-derdewereldland-zou-gebeuren-zouden-de-rapen-gaar-zijn/; https://www.ninefornews.nl/europese-hoofd-van-bilderberg-victor-halberstadt-ontkent-bij-bilderberg-dat-hij-victor-halberstadt-is/; https://www.ninefornews.nl/coronacriticus-van-het-eerste-uur-rashid-buttar-vlak-voor-zijn-dood-ik-ben-doelbewust-vergiftigd/; https://www.ninefornews.nl/acteur-epstein-eiland-is-niet-het-enige-eiland-waar-ze-kinderen-pijn-hebben-gedaan/; https://www.ninefornews.nl/kamerlid-haalt-fel-uit-naar-d66-die-partij-is-een-dogmatische-sekte/; https://www.ninefornews.nl/advocaat-zegt-groenlinks-lidmaatschap-op-omdat-klaver-en-halsema-aangesloten-zijn-bij-world-economic-forum/; https://www.ninefornews.nl/great-reset-architect-yuval-harari-glipt-via-achterdeur-naar-binnen-bij-bilderberg/; https://www.ninefornews.nl/slachtoffers-prikschade-gaan-langs-bij-ministerie-van-vws-wil-je-meedoen-pak-je-kans/; https://www.ninefornews.nl/rutte-en-kaag-krijgen-bakken-kritiek-om-deelname-bilderbergconferentie-zij-zijn-bij-ons-in-dienst/; https://www.ninefornews.nl/rutte-maakt-zich-niet-populair-u-wilt-gewoon-escaleren/; https://www.ninefornews.nl/sucharit-bhakdi-hoe-rna-prikken-je-permanent-genetisch-transformeren/; https://www.ninefornews.nl/veiligheidschef-radioactieve-wolk-drijft-richting-west-europa/; https://www.ninefornews.nl/video-elon-musk-geeft-journalist-een-lesje-in-complottheorieen/; https://www.ninefornews.nl/topacteur-komt-met-baanbrekende-film-over-kinderhandel-er-komt-een-grote-storm-aan-en-dat-weten-ze/; https://www.ninefornews.nl/kijk-rutte-en-kaag-onder-politiebegeleiding-naar-schimmige-bilderbergconferentie/; https://www.ninefornews.nl/bilderberg-2023-van-start-deze-nederlanders-zijn-van-de-partij/; https://www.ninefornews.nl/volgens-president-zelenski-wordt-alles-op-de-krim-kapotgeschoten-zo-is-de-situatie-echt/; https://www.ninefornews.nl/kijk-deze-explainer-tristate-city-was-toch-een-complottheorie/; https://www.ninefornews.nl/de-onderschatte-gezondheidsvoordelen-van-medicinale-cannabis/; https://www.ninefornews.nl/financieel-expert-over-nieuwe-pensioenwet-dat-is-echt-een-kulverhaal-er-zit-een-andere-agenda-achter/; https://www.ninefornews.nl/overgrote-deel-nederlanders-heeft-totaal-geen-idee-van-wat-zich-op-dit-moment-onder-hun-neus-af-aan-het-spelen-is/; https://www.ninefornews.nl/uitbreiding-sleepwet-maakt-nederland-surveillance-staat/; https://www.ninefornews.nl/kolonel-niemand-kan-een-hypersonische-raket-neerhalen-dat-is-gewoon-absurd/; https://www.ninefornews.nl/het-social-media-complot-van-de-jonge-wat-kan-die-man-uniek-makkelijk-liegen-angstaanjagend-gewoon/; https://www.ninefornews.nl/musk-haalt-uit-naar-magneto-george-soros-hij-haat-de-mensheid/; https://www.ninefornews.nl/hier-komt-de-geheimzinnige-bilderberggroep-deze-week-bijeen-en-rutte-is-erbij/; https://www.ninefornews.nl/vader-van-elon-musk-doet-opmerkelijke-uitspraken-over-ontvolkingsplannen-bill-gates/; https://www.ninefornews.nl/wereldwijd-evenement-tegen-de-globalisten-op-20-mei-we-zijn-er-klaar-voor-kom-maar-op/; https://www.ninefornews.nl/aartsbisschop-pleit-voor-verbond-tegen-davos-lobby-en-world-economic-forum-we-voeren-een-historisch-gevecht/; https://www.ninefornews.nl/npo-verspreidde-complottheorie-en-martin-bosma-wilde-een-debat-kijk-naar-de-reacties/; https://www.ninefornews.nl/beelden-klimaatactivisten-roepen-op-de-uva-op-tot-geweld-en-vermoorden-van-joden/; https://www.ninefornews.nl/houdt-kuipers-modellen-en-data-van-rivm-achter-om-pandemiewet-door-te-drukken/; https://www.ninefornews.nl/journalistiek-schandaal-van-de-hoogste-orde-maakt-tongen-los-kamerdebat-aangevraagd/; https://www.ninefornews.nl/terwijl-burgers-nauwlettend-worden-gecontroleerd-smijt-dit-kabinet-zonder-controle-met-miljarden/; https://www.ninefornews.nl/kuipers-bevestigt-hier-dat-hij-elke-maatregel-die-hij-zou-willen-gebruiken-alsnog-kan-invoeren/; https://www.ninefornews.nl/vvd-senator-liegt-in-eerste-kamer-vuile-leugenaar-ik-was-daar-mevrouw/; https://www.ninefornews.nl/fdf-voorman-roept-op-prepare-for-battle/; https://www.ninefornews.nl/eerste-kamerlid-stipt-hier-een-pijnlijk-punt-aan-over-de-pandemiewet/; https://www.ninefornews.nl/nooit-vergeten-deze-partijen-hielden-de-mogelijkheid-open-voor-het-opsluiten-van-ongevaccineerden/; https://www.ninefornews.nl/filmpje-klaus-schwab-heeft-skeletten-in-de-kast/; https://www.ninefornews.nl/schandalig-dat-deze-wetten-zo-kort-voor-de-installatie-van-de-nieuwe-eerste-kamer-worden-behandeld/; https://www.ninefornews.nl/de-boodschap-is-duidelijk-mensen-zijn-erin-genaaid/; https://www.ninefornews.nl/columnist-maakt-gideon-van-meijeren-uit-voor-dwergnazi-en-populistische-pisvlek/; https://www.ninefornews.nl/financieel-expert-over-grootste-schandaal-ooit-het-zit-ontzettend-fout-met-de-pensioenfondsen/; https://www.ninefornews.nl/uitvinder-mrna-vaccins-haalt-uit-naar-klaus-schwab-hij-is-een-fascist/; https://www.ninefornews.nl/einde-van-eigen-auto-in-zicht-you-will-own-nothing/; https://www.ninefornews.nl/zo-zien-vrijgegeven-wob-documenten-over-de-bestrijding-van-desinformatie-eruit/; https://www.ninefornews.nl/dit-is-de-meest-dictatoriale-wet-die-ooit-door-een-regering-is-voorgesteld/; https://www.ninefornews.nl/kijk-kuipers-heeft-even-geen-tijd-voor-vraag-over-aansprakelijkheid-bij-vaccinatieschade/; https://www.ninefornews.nl/kaag-onderbroken-op-d66-partijcongres-ik-voel-mij-verraden-door-u/; https://www.ninefornews.nl/senator-over-grooming-van-kinderen-door-vn-who-tijd-om-te-vertrekken-uit-het-paradijs-van-viespeuken/; https://www.ninefornews.nl/hongaarse-premier-vergelijkt-eu-met-hitlers-plannen-voor-wereldheerschappij/; https://www.ninefornews.nl/ernstige-zorgen-om-drinkwatertekort-in-nederland-dit-zit-er-achter/; https://www.ninefornews.nl/omtzigt-maakt-hier-een-belangrijk-punt-over-aansprakelijkheid-bij-vaccinschade/; https://www.ninefornews.nl/europarlementarier-de-boeren-moeten-verdwijnen-uit-europa-en-dit-voedsel-komt-er-voor-terug/; https://www.ninefornews.nl/europarlementarier-de-boeren-moeten-verdwijnen-uit-europa-en-dit-voedsel-komt-er-voor-terug/; https://www.ninefornews.nl/de-jonge-zou-zich-morgen-moeten-melden-bij-de-tweede-kamer-om-onder-ede-te-worden-gehoord/; https://www.ninefornews.nl/omt-lid-onder-vuur-deze-man-moet-je-echt-zijn-registratie-afpakken/; https://www.ninefornews.nl/immunoloog-waarschuwt-er-komt-iets-nieuws-op-ons-af/; https://www.ninefornews.nl/eerste-kamerleden-staan-aan-de-vooravond-van-de-belangrijkste-beslissing-in-hun-leven/; https://www.ninefornews.nl/wordt-deze-voorzitter-van-world-economic-forum-werkgroep-de-nieuwe-directeur-van-twitter/; https://www.ninefornews.nl/vvd-kamerlid-doet-lacherig-over-huisartsen-die-tijdens-corona-werden-gecensureerd/; https://www.ninefornews.nl/brisant-check-martin-bosma-over-de-duistere-verwevenheid-van-d66-met-dit-lobbykantoor/; https://www.ninefornews.nl/van-haga-woest-dit-zijn-toch-de-zaken-die-niet-gezegd-mogen-worden/; https://www.ninefornews.nl/advocaat-we-moeten-ons-verzetten-tegen-de-pandemiewet-en-wel-hierom/; https://www.ninefornews.nl/kijk-europarlementarier-legt-ten-overstaan-van-het-parlement-de-ware-agenda-bloot/; https://www.ninefornews.nl/europarlementarier-doorprikt-keiharde-leugens-eu-stop-met-de-klimaatdictatuur/; https://www.ninefornews.nl/wie-is-er-aansprakelijk-voor-vaccinatieschade-luister-naar-kuipers/; https://www.ninefornews.nl/kamerlid-deelt-flinke-sneer-uit-aan-vvd-partij-van-oplichters-een-oplichtersbende/; https://www.ninefornews.nl/regering-kan-instemmen-met-wetgeving-zonder-dat-parlement-akkoord-is-buitengewoon-problematisch/; https://www.ninefornews.nl/deze-great-food-reset-leidt-in-nederland-tot-schrijnende-taferelen/'

In [10]:
def extract_urls_from_single_string(url_string: str) -> list:
    '''
    Turns one big string of urls seperated by '; ' into a list of urls.
    the '; ' seperator is a result from concatenating lines in PyCharm with ctrl+shift+j

    :param url_string: string of urls
    :return: list of urls
    '''
    urls = list(url_string.split('; '))
    return urls

extract_urls_from_single_string(nine_for_news_list)

['https://www.ninefornews.nl/page/13/',
 'https://www.ninefornews.nl/europarlementarier-geeft-in-3-minuten-3-redenen-waarom-wereldwijd-pandemieverdrag-gevaarlijk-is/',
 'https://www.ninefornews.nl/world-economic-forum-kondigt-zomer-davos-aan-en-hier-vindt-de-bijeenkomst-plaats/',
 'https://www.ninefornews.nl/russische-soldaten-doen-opmerkelijke-uitspraken-over-oorlog-in-oekraine/',
 'https://www.ninefornews.nl/harde-uithaal-naar-groene-gek-timmermans-hele-gevaarlijke-man/',
 'https://www.ninefornews.nl/zo-hebben-ze-de-pandemie-nog-2-jaar-gerekt-dit-is-een-groot-schandaal/',
 'https://www.ninefornews.nl/brisante-onthullingen-de-hufterigheid-van-rutte-cum-suis-kent-geen-grenzen/',
 'https://www.ninefornews.nl/kaag-kiest-er-bewust-voor-om-de-nederlandse-bevolking-armer-te-maken/',
 'https://www.ninefornews.nl/verslagenheid-na-aannemen-dictatoriale-pandemiewet-moeilijke-moeilijke-tijden-komen-eraan/',
 'https://www.ninefornews.nl/instagram-influencer-verheerlijkt-euthanasie-heb-je-er-zin-i

## Dataframes

In [2]:
df_nine_for_news = pd.read_csv('results_nine_for_news.csv')
df_nine_for_news

,text,date,url
0,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">25 mei ...",https://www.ninefornews.nl/page/13/
1,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">24 mei ...",https://www.ninefornews.nl/europarlementarier-...
2,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">24 mei ...",https://www.ninefornews.nl/world-economic-foru...
3,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">24 mei ...",https://www.ninefornews.nl/russische-soldaten-...
4,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">24 mei ...",https://www.ninefornews.nl/harde-uithaal-naar-...
5,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">24 mei ...",https://www.ninefornews.nl/zo-hebben-ze-de-pan...
6,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">24 mei ...",https://www.ninefornews.nl/brisante-onthulling...
7,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">24 mei ...",https://www.ninefornews.nl/kaag-kiest-er-bewus...
8,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">23 mei ...",https://www.ninefornews.nl/verslagenheid-na-aa...
9,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,"[<span class=""date meta-item tie-icon"">23 mei ...",https://www.ninefornews.nl/instagram-influence...


In [3]:
df_gedachtenvoer = pd.read_csv('results_gedachtenvoer.csv')
df_gedachtenvoer

,text,date,url


In [4]:
df_indymedia= pd.read_csv('results_Indymedia.csv')
df_indymedia

,text,date,url
0,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,"[', gepost door: Amsterdamse Autonome Coalitie...",https://www.indymedia.nl/
1,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,"[', gepost door: Joke Kaviaar op 19/05/2023 05...",https://www.indymedia.nl/node/53410
2,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,"[', gepost door: nn op 17/05/2023 05:09:59']",https://www.indymedia.nl/node/53399
3,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,"[', gepost door: nn op 13/05/2023 05:10:10']",https://www.indymedia.nl/node/53386
4,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,"[', gepost door: Anarchist Black Cross Dresden...",https://www.indymedia.nl/node/53377
...,...,...,...
134,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,[],https://www.indymedia.nl/event/2023/05/27
135,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,"[', gepost door: Internationale Kommunistische...",https://www.indymedia.nl/taxonomy/term/4784
136,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,"[', gepost door: Internationale Kommunistische...",https://www.indymedia.nl/taxonomy/term/4785
137,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,"[', gepost door: Internationale Kommunistische...",https://www.indymedia.nl/node/add/comment/53419


In [5]:
df_niburu = pd.read_csv('results_niburu.csv')
df_niburu

,text,date,url
0,Toggle navigationInfo Het is werkelijk ziekmak...,"[<time datetime=""2023-05-24T13:47:53+02:00"" it...",https://niburu.co/
1,Toggle navigationInfo\n\n\n\n\n\n\n \nD66 kru...,[],https://niburu.co/info/over-ons
2,Toggle navigationInfo \n\n\n\n\n\n\n \nD66 kr...,[],https://niburu.co/info/adverteren
3,Toggle navigationInfoVoor technische vragen ov...,[],https://niburu.co/info/contact
4,Toggle navigationInfo Niburu is afhankelijk va...,[],https://niburu.co/doneren
...,...,...,...
84,Toggle navigationInfo Enkele jaren geleden zij...,"[<time datetime=""2023-05-24T06:04:52+02:00"" it...",https://niburu.co/site-mededelingen/18580-de-w...
85,Ministers liegen alsof het gedrukt staat en de...,"[<time datetime=""2023-05-23T19:02:42+02:00"" it...",https://niburu.co/binnenland/18583-steeds-meer...
86,Toggle navigationInfoDe nep coronapandemie lig...,"[<time datetime=""2023-05-14T11:00:34+02:00"" it...",https://niburu.co/het-complot/18555-tijdlijn-n...
87,"Toggle navigationInfoDit is het niet, omdat he...","[<time datetime=""2021-07-19T20:20:57+02:00"" it...",https://niburu.co/binnenland/16500-het-parleme...


In [6]:
df_privacynieuws = pd.read_csv('results_privacynieuws.csv')
df_privacynieuws

,text,date,url
0,StartNieuwsoverzichtContactNieuwsarchiefBinnen...,"[<time datetime=""2023-05-24T21:25:53+02:00"" it...",https://privacynieuws.nl/
1,StartNieuwsoverzichtContactNieuwsarchiefBinnen...,[],https://privacynieuws.nl/nieuwsoverzicht.html
2,StartNieuwsoverzichtContactNieuwsarchiefBinnen...,[],https://privacynieuws.nl/contact.html
3,StartNieuwsoverzichtContactNieuwsarchiefBinnen...,"[<time datetime=""2019-08-20T17:37:57+02:00"" it...",https://privacynieuws.nl/nieuwsarchief.html?ca...
4,StartNieuwsoverzichtContactNieuwsarchiefBinnen...,"[<time datetime=""2023-05-20T20:38:26+02:00"" it...",https://privacynieuws.nl/binnenlands-nieuws.html
...,...,...,...
343,StartNieuwsoverzichtContactNieuwsarchiefBinnen...,"[<time datetime=""2021-12-08T16:32:47+01:00"" it...",https://privacynieuws.nl/lichamelijke-integrit...
344,StartNieuwsoverzichtContactNieuwsarchiefBinnen...,"[<time datetime=""2021-11-03T15:10:58+01:00"" it...",https://privacynieuws.nl/lichamelijke-integrit...
345,StartNieuwsoverzichtContactNieuwsarchiefBinnen...,"[<time datetime=""2021-10-19T12:02:36+02:00"" it...",https://privacynieuws.nl/lichamelijke-integrit...
346,StartNieuwsoverzichtContactNieuwsarchiefBinnen...,"[<time datetime=""2021-09-17T09:46:53+02:00"" it...",https://privacynieuws.nl/lichamelijke-integrit...
